# Airplane allocation - A stochastic problem

## Problem definition

In a world where everything move fast, airline companies have to adapt their airplane allocation in order to fit as best as they can the passanger demand. To reach that goal, one of the possible method is using a stochastic approach. In fact, many variables must be digest to find one the best possible airplane distribution. A stochastic approach is well fitted for our case because it's allow to search one possible solution among a large set of possible one by trying a lot of airplanes distribution solutions.

In our proble, an airline company wishes to allocate airplanes of various types among its route to satisfy an uncertain passenger demand, in such a way as to minimize operating costs plus the lost revenue from passengers turned away.
This problem use *4* different types of aircraft and *5* routes.

This problem is taken from the book *NUmerical Technics for Stochastic Optimizations* by Yu. Ermoliev & R. J-B Wets (p.565, A.J. King). http://pure.iiasa.ac.at/id/eprint/3065/1/XB-88-403.pdf#page=565

In the following report, we will introduce us our work and our implementation of this airplane allocation problem

## The modeling
In order to modelize the problem, we use a matrix $A$ containing the number of aircraft use for a given route and a given type :

<br>
<center>$A = (x_{i,j})_{1\leq i\leq 5, 1\leq j\leq 4} = \left( \begin{array}{cccc}
x_{1,1} & \ldots & \ldots & x_{1,4} \\
x_{2,1} & \ldots & \ldots & x_{2,4} \\
x_{3,1} & \ldots & \ldots & x_{3,4} \\
x_{4,1} & \ldots & \ldots & x_{4,4} \\ 
x_{5,1} & \ldots & \ldots & x_{5,4} \\ \end{array} \right) \in M_{5,4}(\mathbb{N})$ where $x \in \mathbb{N}$</center>

Where $x_{i,j}$ represents the number of airplane for the type $j$ on the route $i$.

Example $x_{3,4} = 23$ means that we have 23 aircrafts of type 4 on the route number 3

## The cost function

In order to find the best solution, we need to minimize the following cost function :

<br>
<center>$f = \sum_{i=1}^{5} \sum_{j=1}^{4}( Cost_{i,j} * x_{i,j})+ \sum_{k=1}^{5} RevenueLost_{k} * PassengersTA_{k}$</center>

Where :

* $Cost_{i,j}$ characterize the operational cost of an aircraft by the route.
For our implementation, we chose to use this matrix $C \in $ of operational costs :

<center>$C = \left( \begin{array}{cccc}
12 & 20 & 20 & 19 \\
2 & 34 & 10 & 20 \\
43 & 63 & 40 & 12 \\
32 & 10 & 6 & 34 \\ 
20 & 30 & 10 & 87 \\ \end{array} \right)\in M_{5,4}(\mathbb{N})$</center>

Example : $Cost_{1,4} = 19$ and $Cost_{5,2} = 30$

* $RevenueLost_{k}$ with $k \in [1,5]$ denote the revenue lost per passenger turned away on the $kth$ route.
For our implementation, we chose to use this matrix $R$ :

<center>$R = \left( \begin{array}{cccc}
13 & 20 & 7 & 7 & 15\\ \end{array} \right)\in M_{1,5}(\mathbb{N})$</center>

* $PassengersTA_{k} \in \mathbb{N} = D_{1,k} - \sum_{j=1}^{4}(A_{k,j}*B_{k,j})$ with $k \in [1,5]$ denote the number of passenger who turned away from the $kth$ route. This is obtained by substracting the route demand by the real number of passenger knew by multiplying the number of aircraft per route by their capacity. The demand is denote by this matrix $D$:

<center>$D = \left( \begin{array}{cccc}
800 & 900 & 700 & 650 & 380\\ \end{array} \right)\in M_{1,5}(\mathbb{N})$</center>

And the total capacity of each type of aircraft is denoted by the following matrix $B$ : 

<center>$B = \left( \begin{array}{cccc}
16 & 10 & 30 & 23 \\ \end{array} \right)\in M_{1,4}(\mathbb{N})$</center>



**Our goal here will to lower this function in order to fit the demand with the lowest possible cost. To achieve this goal, we choose two approach. The first one is the resoltion by a population based algorithm, the genetic one.**

# Introduction

## Goal
The main goal of our project is to determine which of the both stochastic approach (**simulated annealing or genetic**) is the best fitted for this kind of problem. In order to determine that, we established a test protocol by measuring the number of cost function calls of each algorithm with a set a predifined parameters. Both algorithm are quite different and don't take the same input paramaters. So we choose the best suited for each.

In the following chapters, we will present you how work the **simulated annealing** and the **genetic**, the problems we encountered during the implementations and finally the tests with the results.

First of all, we will introduce in depth those two algorithms, then the researsh we did to determine the best arguments for our test runs.
Finnaly, we will compare the both algorithms and conclude

## First propposed solution - Population based algorithm
The main idea behind this type of algorithm is to generate multiple generation and for each one pick the two best members, cross them, mutate the resulted child and generate a new popuation :

### Key features

* **Population size** : We chose to use a single population. The size is kept constant across all generations
* **Mixing operations** : To generate our children, we  cross the columns between parental matries. When our children are generated, we select one pof them with the unifrom probability distribution. Finally, the picked child is mutated by re-generating the entire given column //DESCRIBE SOMEWHERE
* **Succession scheme** : We use the *µ,µ* succession scheme. This means that with *µ* parents we get *µ* children in our new generation
* **Stopping condition** : As we are looking for the one of the best solution, our stopping condition is determined by the length of our generation. 

### The algorithm

1. Generate a base population $P = \{p_0,...,p_n\}$ of size $n$. The first iteration will generate a random one with one constraint, we have a limited number $T_{1,k}$ of each type $k$ of aircraft : $T = \left( \begin{array}{cccc}
10 & 19 & 25 & 16\\ \end{array} \right)\in M_{1,5}(\mathbb{N})$
2. Compute the total cost of the generation with the our formula previously define.
4. Compute the probality for each individuals to picked them regarding their cost : $P \in [0,1] = \begin{eqnarray}\frac{1/Cost}{\sum_{}1/Cost} \end{eqnarray}$
3. Pick the to best members $\{p_1,p_2\}$ of the population according to their probability. They will be our two *parents*.
4. Cross both parents according to the continuous uniform distribution $unif(0,1)$  and a threshold $C_T \in [0,1]$. For each column of the parents, if $v \leq C_T$ we swap the colum. At the end we have a new child. This is done two times to get two children
5. Pick one of the two children according to the continuous uniform distribution $unif(0,1)$, if $unif(0,1) \geq 0.5$ we pick the first child, then we pick the second.
6. Mutate the picked child according to the continuous uniform distribution $unif(0,1)$ and a threshold $M_T \in [0,1]$. For each column of the child, if $v \leq M_T$ we swipe two random value in the same row.
7. We repeat *4, 5 and 6* until the size of the new generation is equal to the previous one ($n$).

### Set of research

Let's denote $(\Omega, \mathcal{F}, \mathcal{P})$ our probalistic space : 

<center>$\Omega=\{M\in M_{5,4}(\mathbb{N}) ; \sum_{k=1}^{5}M_{k,n}=T_{1,n} , n\in[1,4]\}$</center>
<br>
<center>$\mathcal{F}=2^\Omega$</center>
<br>
<center>$Probality\ on\ M\ is\ the\ particular\ [0,1] \ni Pr(M) = \begin{eqnarray}\frac{1/Cost(M)}{\sum_{}1/Cost(M)} \end{eqnarray}\ (cf. Cost\ function\ paragraphe)$<center>

## Encountered problems - The non convergence

The first implementation of the genetic algorithm was based as described in the previous part on probabilities. In fact, in order to choose our parents, we computed the cost of each one and then associate the corresponding probability. At the end, the parents was picked up following their associated probabilty.

This graph shows you for every generation, the best cost we can get. In the run, we run *400* generations of *30* individuals.

<img src="assets/images/1_airplanes_g400_i30_cT0.3_mT0.15.png">


Unfortunately as you can see, the result is not converging toward a possible solution but it is chaotic. This is because of the way we pick our parents. We try in a first place to  If we base this on the probabilities, the results are not good.

<br>
<center>In this first run, $C_{T} = 0.3$ and $M_{T}=0.15$ and $min_{cost}=20660\$$</center>

In order to get around this problem, we decided to change the way of picking the parents responsible to generate the future generation. Instead of picking them according to there probability, we directly pick the two best element of the population.
So we change the fourth step of the algorithm by this : 

**4. Pick the to best members $\{p_1,p_2\}$ of the population according to their cost. The parents with the lowest cost among all the population. They will be our two *parents*: $p_1 = \underset{x \in P}{min}\ f(x) \quad \textrm{and} \quad p_2 = \underset{x \in P-{p_1}}{min}\ f(x)$.**

<img src="assets/images/2_airplanes_g400_i30_cT0.3_mT0.15.png">


As you can see we have a covergence toward one of the possible solution and no more chaos.

<br>
<center>In this second run,  $C_{T} = 0.3$ and $M_{T}=0.15$ and $min_{cost}=18980\$$</center>

## Parameters optimization

In order to pick the apropriates arguments for the genetic stochastic approach, I run multiple times a a range of simulation with differents parameters :

### Statistic representation

<br>
<center><b>(vertical = number of generation ; horizontal = population size) </b></center>

<img src="assets/images/matplotlib-table_cT=0.4 & mT=0.15 (40 runs).png">

<img src="assets/images/matplotlib-table_cT=0.5 & mT=0.17 (40 runs).png">

<img src="assets/images/matplotlib-table_cT=0.6 & mT=0.2 (40 runs).png">

<img src="assets/images/matplotlib-table_cT=0.6 & mT=0.15 (40 runs).png">

### Decision

<center><b>As you can see, the last runs with $C_{T}=0.6$ and $M_{T}=0.2$ gave us the best result</b></center>

### Evaluation

When we run the algorithm with the following arguments, we get this graph :

<br>
<center>
$n = 160\ p = 80\ C_{T}=0.6\ M_{T}=0.2$
</center>
<img src="assets/images/3_airplanes_g80_i160_cT0.6_mT0.2.png">

As we can see, there is a very quick convergence

## Second proposed solution - Simulated Annealing 

The simulated annealing is a good choice when one is trying to find a single best solution to a problem, and this is the case with this aircraft allocation problem.

### Key features

* **Temperature** : The main parameters for the simulated annealing algorihtm, the temperature allows this algorithm to have a lot of diversity at the beginning to finally converge to a unique solution.
* **Decrease Factor** : At each end of a loop the temperature is multiplied by (1 - Decrease Factor), thus the decrease factor define the speed of convergence of this algorithm.
* **Number of neighbors generated each time** : To implement a successful S.A. algorithm we needed to generate some neighbors from a given solution to create diversity and thus help the algorithm to find the best solution. The number of neighbors generated each time will remain the same : 4.
    
### The algorithm

#### Pseudo code 

<img src="assets/images/sa_algo.png" width=60% height=60%>

RandomProbability is a number randomly generated between 0 and 1.

#### Neighbors generation

Let's explain more pricesely how the neighbords are generated. First, as displayed in the pseudo code, there is not only 1 step in the S.A. algorithm before the decreasing of the temperature but 4. We choose this to simplify the neighbors generation and to modify airplane allocation one airplane type at a time, so as there are 4 airplane type it is done 4 times.  
  
There are 3 operations to generate neighbors, and they all keep the same amount of plane as constraint :

* Single Random Permutation
    * Ex : From (2,**1**,3,0,**4**) to (2,**4**,3,0,**1**) ( 4 and 1 permuted)     

* Random +1 / -1 
    * ex : from (2,1,3,**0**,**4**) to (2,1,3,**1**,**3**)   
 
* Whole New Column
    * Ex : from (2,1,3,0,4) to (3,0,3,2,2) (whole new allocation)   
  
Thanks to this 3 operations the S.A. algorithm is capable of creating enough diversity to converge towards a better and better candidate.  


## Parameters Optimization

The algorithm work as expected, however we also need to find the good set of parameters for our problem in order to have the best result in the minimum time.

### Result

Here is some result with the cost of the current candidate in ordinate and the number of loop in the while loop in absciss.

<img src="assets/images/sa_99_01.png">

<img src="assets/images/sa_999_01.png">

<img src="assets/images/sa_9999_01.png">


### Some statistical datas

In order to understand in a more effective way how the Temperature and the decrease factor influences the result, we realize some statistical computation. There is below some data using 200 output of the algorithm per case (so the SA algorithm ran 3200 times for these data). In green the minimum value and in red the maximum value.

<img src="assets/images/sa_datas.png">

### Decision

In the statictical data we can see that (99,0.1) is obviously the worst solutions as the algorithm has not enough time to create diversity in order to find the best solutions and move away from local minimum. Looking at the mean and the median the couple (999,0.01) seems to be pretty good, but if we look at the first and third quartile it is possible to see that (9999,0.01) and (9999,0.0001) seem to also give good result. Finally, even if (9999,0.01) and (9999,0.0001) seem to be good candidate, (9999,0.01) takes more than twice loop than (999,0.1) and (9999,0.0001) more than ten times.  

***So the best set of parameters for the Simulated Annealing is Temperature = 999 and DecreaseFactor = 0.01***.

## Solutions evaluation  

#### Intro  
  
In order to compare the Genetic Algorithm and the Simulated Annealing solutions, it has been decided to compare them using the same element wich will represent the cost of each solutions : the number of times the cost evalution is done. In the code is will be how many times the cost evaluations function is called. 

### Simulated Annealing

As the cost of the algorithm was defined as the number of times the cost evalution is done, here is the mean value and standard deviation for the S.A. algorithm with the previous set of parameters (999,0.01). The evalution is done by counting the number of times the cost evalution is done until the algorithm find a candidate wich has a cost $<= 18 700$. For these values the S.A. algorithm ran more than 300 times.
  
***S.A. cost, Mean Value : 3225   
S.A. cost, Standard Deviation : 1582***  
  
This numbers may seem big, however they don't represent the number in absciss showed above in the Result section. In the graphics this is the number of loop in the While Loop and in each While Loop 4 new candidate are generated 4 times. So in each whille loop the cost evaluation is done 16 times. So if we divided these number by 16 (Mean Value : 201 / Standard Deviation : 98 ), we have more relevant numbers to compare with the graphic. 
  
### Genetic Algorithm

As we can see in the differents runs, the best mean value we can get is around 19100. So in order to determine the number of cost function call, I will look how many generations does it take to get get a cost below 19200 with the following parameters : 

<br>
<center>
$n = 160\ p = 80\ C_{T}=0.6\ M_{T}=0.2$
</center>

<img src="assets/images/time_matplotlib-table_cT=0.6 & mT=0.2 (40 runs).png">

If we compare these values with the number of the simulated annealing approach, we get much fewer calls. Even if we get a slightly lower cost with the first approach **(18700 vs 19200)**, the genetic one if much cheaper in term of computing cost.



# Conclusion

The airline sector his one of the most competitive in the world. Because of that, every cost that may be cut have to. In order to do that, many solutions to evaluate the profitability of a model exist. In our, we choose to evaluate a given model using two stochastics algorithm : Simulated Annealing and the genetic population based. From that, we decided to evaluate which of both may be the best fitted for our needs which are being able to get an aircraft distribution with the lowest cost possible. 

In order to do that, we run with both methods multiple simuation in order to pick the best parameters for each implementation. When these arguments has been choosen, we fixed then and run the final simulation and counted how much the cost function has been called during the computation. The lower this number, the more effective the algorithm is. 

We get these results : 

<center>
    $Simulated\ annealing=201$
<center>
<br>
<center>
    $Genetic\ population\ based=74\ (mean\ of\ 40\ runs)$
</center>
    
It can be seen that the genetic population based algorithm require fewer call to the cost function thant the simulated annealing one. However, it's important to underline that the second methods one get slightly better results (18652) thant the first one (18838)
    
To conclude, we can say that our researches brought out that **the best algorithm to pick in term of computational cost is the genetic** one. However its important to highlight that we get a slightly lower cost with the simulated annealing and also that other stochastic approach may have be better (ant colony for example)